In [2]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))

house_prices <- source('main.R', local = TRUE)$value

################################
# load data, missing, outliers #
################################
combined_dataset_step1 <- 
    house_prices$helpers$load_data() %>%
    (house_prices$stage1_transformation)

################################
# transform numeric predictors #
################################
step2 <- house_prices$stage2_transformation(
    combined_dataset_step1,
    threshold = 20
)
trans_config <- step2$trans_config
combined_dataset_step2 <- step2$dataset
                
trans_config

house_prices$helpers$utils$frames_diff(
    combined_dataset_step1, 
    combined_dataset_step2
) %>% head

####################################                
# transform categorical predictors #
####################################
combined_dataset_step3 <- house_prices$stage3_transformation(combined_dataset_step2)

house_prices$helpers$utils$frames_diff(
    combined_dataset_step2, 
    combined_dataset_step3
) %>% head

var,tran_name,progress_score,tran_fn,r2_x,r2_tran
GrLivArea,log,91.15794,"function (x) , log(x + 1)",0.52593095,0.5438044
LotArea,inv4,78.86403,"function (x) , x^(1/4)",0.06788313,0.1591873
TotRmsAbvGrd,log,30.68502,"function (x) , log(x + 1)",0.28912299,0.2938676


GrLivArea.1,GrLivArea.2,LotArea.1,LotArea.2,TotRmsAbvGrd.1,TotRmsAbvGrd.2
1710,7.444833,8450,9.587694,8,2.197225
1262,7.141245,9600,9.898464,6,1.945910
1786,7.488294,11250,10.298836,6,1.945910
1717,7.448916,9550,9.885550,7,2.079442
2198,7.695758,14260,10.927728,9,2.302585
1362,7.217443,14115,10.899842,5,1.791759


Alley.1,Alley.2,BldgType.1,BldgType.2,BsmtCond.1,BsmtCond.2,BsmtExposure.1,BsmtExposure.2,BsmtFinType1.1,BsmtFinType1.2,⋯,RoofStyle.1,RoofStyle.2,SaleCondition.1,SaleCondition.2,SaleType.1,SaleType.2,Street.1,Street.2,Utilities.1,Utilities.2
_none_,2.531822,1Fam,2.564039,TA,2.540871,No,2.320042,GLQ,3.262019,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Gd,3.295455,ALQ,2.231818,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Mn,2.675439,GLQ,3.262019,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,Gd,3.046154,No,2.320042,ALQ,2.231818,⋯,Gable,2.418054,Abnorml,1.920792,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Av,2.900452,GLQ,3.262019,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,No,2.320042,GLQ,3.262019,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911


In [66]:
df <- combined_dataset_step3 %>% select(-Id, -price_log, -dataSource)

M <- abs(cor(df))
diag(M) <- 0
M <- as_data_frame(M)

cols <- purrr::map_lgl(M, function(col) { any(col > 0.8) })

M2 <- M[names(which(cols))]

M2

Exterior1st,Exterior2nd,FireplaceQu,Fireplaces,GarageArea,GarageCars,GarageCond,GarageQual,GarageYrBlt,GrLivArea,SaleCondition,SaleType,TotRmsAbvGrd
0.18022485,0.16957717,0.093600809,0.089548323,0.110114946,0.11523502,0.164501768,0.17193702,0.106612230,0.024138273,0.074619842,0.06107683,0.02152246
0.02140594,0.02574136,0.086147347,0.086752830,0.073646538,0.09258671,0.013994206,0.01243333,0.037618831,0.532219450,0.063173476,0.04252915,0.67758484
0.09107737,0.11963063,0.195366943,0.163908643,0.131585086,0.12269529,0.218002269,0.17883578,0.247040668,0.007386022,0.122310670,0.09589607,0.10487825
0.16614378,0.15835077,0.109334231,0.098928081,0.159486152,0.16638180,0.195284197,0.17905952,0.151217442,0.087178901,0.107853357,0.11050195,0.01487242
0.19663115,0.20533288,0.192135853,0.217458232,0.267261934,0.26058719,0.109845120,0.12595076,0.093896474,0.145253077,0.133869719,0.14922779,0.02414186
0.12583715,0.11492941,0.230580687,0.285834315,0.299181948,0.25857588,0.164544551,0.19631406,0.127039797,0.164216664,0.037285052,0.04400765,0.03237901
0.05351425,0.08158457,0.002075769,0.066307393,0.004094678,0.01428972,0.061050560,0.07331030,0.048806009,0.022210666,0.069453840,0.07646463,0.04873159
0.36529609,0.37674494,0.197611889,0.135105360,0.325880939,0.35169358,0.115807476,0.12945501,0.106906988,0.230932496,0.188194781,0.19266732,0.07780351
0.11899838,0.11603613,0.084158205,0.058686091,0.086250690,0.09795283,0.069583209,0.05604140,0.067546818,0.092414834,0.079437038,0.07539269,0.02205675
0.08328502,0.08507738,0.120254672,0.165672394,0.180963633,0.16175839,0.095731568,0.11922045,0.063196652,0.043425442,0.006286964,0.01324913,0.05074743
